# Integrating with OPERA's ISCE3 s1-reader

This notebook requires `isce3`, and `s1-reader` to be installed as well as the rest of the packages listed in `requirements.txt`. Install using these commands:
```bash
conda install -c conda-forge isce3
conda install -c conda-forge backoff

git clone https://github.com/opera-adt/s1-reader.git
python -m pip install -e s1-reader
```

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import xml.etree.ElementTree as ET
import datetime

import isce3
import s1reader
from s1reader import Sentinel1BurstSlc
import bursts

In [2]:
INTERNATIONAL_IDS = {'S1A': ' 2014-016A', 'S1B': '2016-025A'}
zip_path = Path('../example_data/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip')
url_path = 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip'
safe_name = zip_path.with_suffix('.SAFE').name
swath_num = 1
pol = "VV"

## Opera Workflow

In [3]:
# read orbits
orbit_dir = Path('../example_data/sentinel1_orbits')
if not orbit_dir.is_dir():
    orbit_dir.mkdir()

In [4]:
s1reader.s1_orbit.download_orbit(safe_name,orbit_dir)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [5]:
orbit_path = s1reader.get_orbit_file_from_dir(zip_path, orbit_dir)

In [6]:
# returns the list of the bursts
opera_burst_items = s1reader.load_bursts(zip_path, orbit_path, swath_num, pol)
opera_burst_ids = [x.burst_id for x in opera_burst_items]
opera_burst = opera_burst_items[0]

## Now with ASF package

- orbit is calculated using orbit file s1_reader:132:get_burst_orbit
-

In [5]:
# auth = bursts.get_netrc_auth()
# manifest, annotations = bursts.edl_download_metadata(url_path, auth)

In [7]:
manifest, annotations = bursts.local_read_metadata(zip_path)

In [9]:
slc = bursts.SLCMetadata(url_path, manifest, annotations)
swath = bursts.SwathMetadata(slc, pol, swath_index=swath_num - 1)
asf_burst_list = [bursts.BurstMetadata(swath, x) for x in range(swath.n_bursts)]
asf_burst = asf_burst_list[0]
item = asf_burst.to_stac_item()

In [11]:
asf_opera_burst = bursts.stac_item_to_opera_burst(item,'VV',orbit_dir)

In [12]:
asf_opera_burst

Sentinel1BurstSlc(sensing_start=datetime.datetime(2021, 1, 31, 15, 15, 56, 832500), radar_center_frequency=5405000454.33435, wavelength=0.05546576, azimuth_steer_rate=0.027757171601738514, azimuth_time_interval=0.002055556299999998, slant_range_time=0.005354305379666438, starting_range=802590.1853264123, iw2_mid_range=878040.0430978123, range_sampling_rate=64345238.12571428, range_pixel_spacing=2.329562114715323, shape=(1502, 21209), azimuth_fm_rate=Poly1d(order= 2, mean=802590, std=1.49896e+08), doppler=Doppler(poly1d=Poly1d(order= 2, mean=803664, std=1.49896e+08), lut2d=<isce3.ext.isce3.core.LUT2d object at 0x190a3d430>), range_bandwidth=56500000.0, polarization='VV', burst_id='t174_372322_iw1', platform_id='S1B', center=(-132.9122527111637, 57.55822019389534), border=[(-132.166549306571, 57.56074205906996), (-132.2349132599047, 57.56861180376772), (-132.3226205273915, 57.57863487756147), (-132.3960619966416, 57.58698413093212), (-132.4807388585007, 57.5965451025774), (-132.559448877

In [ ]:
properties = item.properties
asset = item.assets[pol]
asset_properties = asset.to_dict()
# platform
platform = [k for k in INTERNATIONAL_IDS if INTERNATIONAL_IDS[k] == properties['sat:platform_international_designator']][0]

# doppler
shape = (asset_properties['lines'], asset_properties['samples'])
doppler_poly1d = isce3.core.Poly1d(*properties['doppler'])
doppler_lut2d = s1reader.s1_reader.doppler_poly1d_to_lut2d(doppler_poly1d,
                                                           properties['starting_range'],
                                                           properties['range_pixel_spacing'],
                                                           shape,
                                                           properties['azimuth_time_interval'])
doppler = s1reader.s1_burst_slc.Doppler(doppler_poly1d, doppler_lut2d)

# orbit
orbit_path = s1reader.get_orbit_file_from_dir(asset.href.split('/')[-1], orbit_dir)
with open(orbit_path,'r') as f:
    orbit_xml = ET.parse(f)
osv_list = orbit_xml.find('Data_Block/List_of_OSVs')
sensing_duration = datetime.timedelta(seconds=shape[0] * properties['azimuth_time_interval'])
orbit = s1reader.s1_reader.get_burst_orbit(item.datetime, item.datetime + sensing_duration, osv_list)

args = dict(
    sensing_start=item.datetime,
    radar_center_frequency=properties['sar:center_frequency'],
    wavelength=properties['wavelength'],
    azimuth_steer_rate=properties['azimuth_steer_rate'],
    azimuth_time_interval=properties['azimuth_time_interval'],
    slant_range_time=properties['slant_range_time'],
    starting_range=properties['starting_range'],
    iw2_mid_range=properties['iw2_mid_range'],
    range_sampling_rate=properties['range_sampling_rate'],
    range_pixel_spacing=properties['range_pixel_spacing'],
    shape=shape,
    azimuth_fm_rate=isce3.core.Poly1d(*properties['azimuth_frame_rate']),
    doppler=doppler,
    range_bandwidth=properties['range_bandwidth'],
    polarization=pol,
    burst_id=properties['opera_id'],
    platform_id=platform,
    center=properties['center'],
    border=list(item.geometry['coordinates'][0]),
    orbit=orbit,
    orbit_direction=properties['sat:orbit_state'].capitalize(),
    tiff_path=f'{asset.href}/{asset_properties["interior_path"]}',
    i_burst=properties['burst_index'],
    first_valid_sample=properties['first_valid_sample'],
    last_valid_sample=properties['last_valid_sample'],
    first_valid_line=properties['first_valid_line'],
    last_valid_line=properties['last_valid_line'],
    range_window_type=properties['range_window_type'].capitalize(),
    range_window_coefficient=properties['range_window_coefficient'],
    rank=properties['rank'],
    prf_raw_data=properties['prf_raw_data'],
    range_chirp_rate=properties['range_chirp_rate']
)

In [264]:
[x for x in args if not hasattr(opera_burst, x)]

[]

In [265]:
[x for x in args if type(args[x]) != type(getattr(opera_burst, x))]

['center']

In [266]:
oks = ['azimuth_fm_rate','doppler','orbit', 'center', 'border','tiff_path']
[x for x in args if (args[x] != getattr(opera_burst, x)) & (x not in oks)]

[]

In [271]:
asf_to_opera_burst = Sentinel1BurstSlc(**args)

tada!